In [1]:
from pymongo import MongoClient
import pprint
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import requests
from bs4 import BeautifulSoup
import json
import time
import copy

In [2]:
url_list = []
for i in list(range(2009, 2020)):
    string = 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr={}&type=reg&cat=T&group=D&conf='
    string = string.format(i)
    url_list.append(string)
url_list

['https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2009&type=reg&cat=T&group=D&conf=',
 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2010&type=reg&cat=T&group=D&conf=',
 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2011&type=reg&cat=T&group=D&conf=',
 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2012&type=reg&cat=T&group=D&conf=',
 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2013&type=reg&cat=T&group=D&conf=',
 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2014&type=reg&cat=T&group=D&conf=',
 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2015&type=reg&cat=T&group=D&conf=',
 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2016&type=reg&cat=T&group=D&conf=',
 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2017&type=reg&cat=T&group=D&conf=',
 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2018&type=reg&cat=T&group=D&conf=',
 'https://www.footballdb.com/stats/teamstat.html?l

In [3]:
url = 'https://www.footballdb.com/stats/teamstat.html?lg=NFL&yr=2016&type=reg&cat=T&group=D&conf='
headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15",
        "Host": "www.footballdb.com",
        "Referer": "https://www.footballdb.com/stats/teamstat.html?group=D&cat=T"
    }
r = requests.get(url,headers=headers)

In [4]:
soup = BeautifulSoup(r.text, 'html.parser')
tables = soup.find('table')
tables

<table class="statistics scrollable">
<thead>
<tr class="header right">
<th class="left"><a href="/stats/teamstat.html?lg=NFL&amp;yr=2016&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=teamname" title="Team">Team</a></th>
<th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2016&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=games" title="Games">Gms</a></th>
<th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2016&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=points" title="Total Points Scored">Tot Pts</a></th>
<th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2016&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=ptsgm" title="Points Per Game">Pts/G</a></th>
<th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2016&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=rushyds" title="Total Team Rushing Yards">RushYds</a></th>
<th><a href="/stats/teamstat.html?lg=NFL&amp;yr=2016&amp;type=reg&amp;cat=T&amp;group=D&amp;conf=&amp;sort=rushypg" title="Rushing Yards Per Game">RYds

In [29]:
col_names = soup.find_all('th')
row_names = soup.find_all('tr')
row_names[1]

<tr class="row0 right">
<td class="left nowrap"><span class="hidden-xs"><a href="/teams/nfl/new-england-patriots">New England Patriots</a></span><span class="visible-xs"><a href="/teams/nfl/new-england-patriots">New England</a></span></td><td>16</td><td>225</td><td>14.1</td><td>1528</td><td>95.5</td><td>2886</td><td>180.4</td><td>4414</td><td class="hilite">275.9</td></tr>

In [19]:
columns = {}
for col in col_names:
    columns[col.text] = None

In [31]:
dict_data = []
keys = list(columns.keys())
for i, row in enumerate(row_names):
    if i > 0:
        new_row = copy.copy(columns)
        entries = row.find_all('td')
        for j, entry in enumerate(entries):
            new_row[keys[j]] = entry.text
        dict_data.append(new_row)
df = pd.DataFrame(dict_data)
df.tail(10)

,Team,Gms,Tot Pts,Pts/G,RushYds,RYds/G,PassYds,PYds/G,TotYds,Yds/G
22,Carolina PanthersCarolina,16,470,29.4,2296,143.5,3696,231.0,5992,374.5
23,Jacksonville JaguarsJacksonville,16,397,24.8,2229,139.3,3778,236.1,6007,375.4
24,New York GiantsNY Giants,16,451,28.2,1812,113.2,4225,264.1,6037,377.3
25,Seattle SeahawksSeattle,16,398,24.9,1883,117.7,4223,263.9,6106,381.6
26,Washington RedskinsWashington,16,435,27.2,2339,146.2,3823,238.9,6162,385.1
27,Houston TexansHouston,16,385,24.1,1937,121.1,4276,267.2,6213,388.3
28,Cincinnati BengalsCincinnati,16,420,26.2,2382,148.9,3917,244.8,6299,393.7
29,Miami DolphinsMiami,16,494,30.9,2166,135.4,4198,262.4,6364,397.8
30,Detroit LionsDetroit,16,423,26.4,1855,115.9,4551,284.4,6406,400.4
31,Arizona CardinalsArizona,16,442,27.6,1922,120.1,4510,281.9,6432,402.0
